In [1]:
import numpy as np
import pandas as pd

from sklearn import preprocessing

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, cross_val_predict

from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC


from sklearn.model_selection import StratifiedKFold, RepeatedKFold

from scipy.stats import pointbiserialr

In [2]:
# df_data = pd.read_csv('not_hold_out_dataset.csv')
df_data = pd.read_csv('orange_small_churn_train_data.csv', index_col='ID')
df_data.rename(columns={"labels": "Label"}, inplace=True)
df_data.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,Label
ID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,3052.0,NaN,NaN,NaN,NaN,...,vr93T2a,LM8l689qOp,NaN,NaN,fKCe,02N6s8f,xwM2aC7IdeMC0,NaN,NaN,-1.0
1,NaN,NaN,NaN,NaN,NaN,1813.0,7.0,NaN,NaN,NaN,...,6hQ9lNX,LM8l689qOp,NaN,ELof,xb3V,RAYp,55YFVY9,mj86,NaN,-1.0
2,NaN,NaN,NaN,NaN,NaN,1953.0,7.0,NaN,NaN,NaN,...,catzS2D,LM8l689qOp,NaN,NaN,FSa2,ZI9m,ib5G6X1eUxUn6,mj86,NaN,-1.0
3,NaN,NaN,NaN,NaN,NaN,1533.0,7.0,NaN,NaN,NaN,...,e4lqvY0,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN,1.0
4,NaN,NaN,NaN,NaN,NaN,686.0,7.0,NaN,NaN,NaN,...,MAz3HNj,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN,-1.0


In [3]:
df_data = df_data[df_data['Label'].notnull()]

df_data['Label'].replace(to_replace=-1, value=0, inplace=True)

In [4]:
# Список столбцов, в котором все строки Nan
columns_nan = [ c for c in df_data.columns if df_data[c].isnull().all()  ]
# Удаление столбцов
df_data = df_data.drop(columns=columns_nan)

# columns_nan
columns_categ = df_data.select_dtypes(['object']).columns
columns_val = set(df_data.columns).difference(set(list(columns_categ) + ['Label', 'ID']))

# Список столбцов, в котором все значения равны. НО надо вставить проверку на то,
# что это не искомый флаг. Будем удалять столбец, если отношение вероятностей 
# оттока и не оттока равны.
# Наверное, отложим так как лучше в дальнейшем смотртеть через корреляцию.
# columns_const = [c for c in df_data.columns if df_data[c].nunique() == 1]

In [5]:
# Var118 и Var67 содержат одно значение - 3.0 и 0.0 соответвтенно.
# так как при определении коэффициента корреляции черещ pointbiserialr не допустимо
# иметь одно значение, заменим в этих переменных Nan на -1.
# df_data[['Var118', 'Var67']].max()
# df_data[['Var118', 'Var67']].fillna(value=-1, inplace=True)
df_data['Var67'].fillna(value=-1, inplace=True)
df_data['Var118'].fillna(value=-1, inplace=True)

In [6]:
# df_data['Var191'].nunique()
# columns_const
# np.asarray([0, 0, 0, 1, 1], dtype = bool)
# columns_val
# df_data['Var118'].info()

In [7]:
pvalue_border = 0.05
acorr = []
for c in columns_val:
    ser = df_data[['Label',c]].dropna()
    rez = pointbiserialr(ser['Label'], ser[c])
    acorr.append([c, rez.correlation, rez.pvalue])
df_corr_var = pd.DataFrame(acorr, columns=['Var', 'correlation', 'pvalue'], index=None)
df_corr_var['correlation_abs'] = df_corr_var['correlation'].abs()
# Получен список численных переменных, коррелированных с выходным параметром

df_corr_var[df_corr_var['pvalue'] <= pvalue_border].Var.values

array(['Var144', 'Var72', 'Var113', 'Var132', 'Var65', 'Var140', 'Var6',
       'Var81', 'Var13', 'Var119', 'Var126', 'Var74', 'Var73', 'Var143',
       'Var111', 'Var160', 'Var168', 'Var7', 'Var177', 'Var10', 'Var19',
       'Var188', 'Var184', 'Var53', 'Var131', 'Var189', 'Var67', 'Var139'],
      dtype=object)

In [17]:
df_corr_var[df_corr_var['correlation_abs'] > 0.01 ].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129 entries, 0 to 171
Data columns (total 4 columns):
Var                129 non-null object
correlation        129 non-null float64
pvalue             129 non-null float64
correlation_abs    129 non-null float64
dtypes: float64(3), object(1)
memory usage: 5.0+ KB


In [18]:
# columns_val_used = df_corr_var[df_corr_var['pvalue'] <= pvalue_border].Var.values
columns_val_used = df_corr_var[df_corr_var['correlation_abs'] > 0.01].Var.values

columns_val_not_used = set(columns_val).difference(set(columns_val_used))

In [66]:
# df_data[columns_categ].info()

In [19]:
%%time
# так как результат предсказания низкий. Пробуем следующее:
# в категориальных переменных, обучающей выборки оставим только те значения,
# которые есть в тестовой выборке.
df_test = pd.read_csv('orange_small_churn_test_data.csv')
for c in columns_categ:
    enable_val = df_test[c].unique()
    df_data[c] = df_data[c].apply(lambda x: x if x in enable_val else np.NaN )
df_test = None   

Wall time: 16.3 s


In [68]:
# # df_data[columns_categ].info()
# df_test = pd.read_csv('orange_small_churn_test_data.csv')
# print(df_test['Var215'].unique())
# print(df_data['Var215'].unique())

In [20]:
%%time
# Вычисление корреляции каждой переменной в каждом категориальном параметре
# Цель - в дальнейшем оставить переменные и значения с наибольшей корреляцией
# или коррелирующиеся.
# Так как расчет выполняется несколько минут (только что - 35 сек.), то данные сохранены в файл
# acorr = []
# for c in columns_categ:
#     df_dummies=pd.get_dummies(df_data[c])
#     for c_val in df_dummies.columns:
#         rez = pointbiserialr(df_data['Label'], df_dummies[c_val])
#         acorr.append([c, c_val, rez.correlation, rez.pvalue])
# # df_data['Var193']
# df_corr_categ = pd.DataFrame(acorr, columns=['Var', 'Val', 'correlation', 'pvalue'], index=None)
# df_corr_categ.to_pickle('df_corr_categ.bin')
df_corr_categ = pd.read_pickle('df_corr_categ.bin')
df_corr_categ['correlation_abs'] = df_corr_categ['correlation'].abs()

Wall time: 151 ms


In [25]:
df_corr_categ[df_corr_categ['correlation_abs'] >= 0.02].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 606 entries, 27 to 15636
Data columns (total 5 columns):
Var                606 non-null object
Val                606 non-null object
correlation        606 non-null float64
pvalue             606 non-null float64
correlation_abs    606 non-null float64
dtypes: float64(3), object(2)
memory usage: 28.4+ KB


In [26]:
# список категориальных полей, которые останутся, и которые должны быть удалены,
# так как их нет в df_corr_categ
# ser_corr_categ = df_corr_categ[df_corr_categ['pvalue'] <= pvalue_border] 
ser_corr_categ = df_corr_categ[df_corr_categ['correlation_abs'] >= 0.02] 

columns_categ_used = ser_corr_categ['Var'].unique()
columns_categ_not_used = set(columns_categ).difference(set(columns_categ_used))
columns_categ_not_used

{'Var191',
 'Var194',
 'Var196',
 'Var203',
 'Var208',
 'Var213',
 'Var215',
 'Var219',
 'Var223',
 'Var224'}

In [27]:
# ограничение на количество столбцов для анализа. Без этого порядка 2 тыс, идет своп
ser_corr_categ = ser_corr_categ.sort_values(by=['correlation_abs']).head(1000)
print(len(ser_corr_categ))

606


In [28]:
# Создание нового массива из необходимых для анализа полей (Можно пойти и наоборот - удалить
# поля, которые не нужны).
### columns_used = list(columns_val_used) + list(columns_categ_used) + ['Label']
df_categ = df_data[columns_categ_used].copy()
# df_categ.info()

In [29]:
# теперь в категориальных полях оставим только коррелирующие значения, заменив остальные на 
for c in df_categ.columns:
    lvals = ser_corr_categ.loc[ser_corr_categ.Var == c].Val.values
    df_categ[c] = df_categ[c].apply(lambda x: x if x in lvals else 'Другое')

In [30]:
df_dummies=pd.get_dummies(df_categ)
df_dummies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18298 entries, 0 to 18297
Columns: 633 entries, Var192_4E_Oj1siQ3 to Var229_Другое
dtypes: uint8(633)
memory usage: 11.2 MB


In [31]:
df_test = pd.concat([df_data[columns_val_used], df_dummies], axis=1)

In [ ]:
# df['Var221'].unique()
# ser_corr_categ.loc[ser_corr_categ.Var == 'Var221'].Val.values

In [ ]:
class Prepare_dataset():
    def __init__(self, border=0.):
        # Список столбцов, в котором все строки Nan
        self.columns_nan = []
        return
    
    def fit(self, df):
        self.columns_nan = [ c for c in df.columns if df[c].isnull().all()  ]

        return
    
    def transform(self, df):
        # Удаление столбцов, в которых все строки Nan
        dfRet = df.drop(columns=columns_nan)
        return dfRet


In [ ]:
# Удалим строки в которых все значения NaN.
# df_tmp = df_data.iloc[:, list(range(0, 190+40)) + [-1]].dropna(axis=1, how='all')
# df_tmp.info()
print(df_data.info())
# df_tmp = df_data.dropna(axis=1, how='all')
df_tmp = df_data.drop(columns=columns_nan)

print(df_tmp.info())

Обработаем категриальные параметры

In [ ]:
# df_tmp.dtypes
# columns[dtype='object']
# df_object = df_tmp.select_dtypes(['object'])

In [ ]:
# Не выполнилось - MemoryError 
# df_dummies=(pd.get_dummies(df_object) > 0).astype(np.int8)

In [ ]:
# df_dummies.to_csv('df_dummies.csv') - бесконечно долго 
# to_pickle('df_dummies.bin')

In [ ]:
# df_object = df_object.astype('category')

In [ ]:
# df_object.info()

In [ ]:
# df_object['Var191'].cat.codes
# df_object=df_object.appply.cat.codes
# df_object.head()
# columns = df_object.columns
# df_object[columns]=df_object[columns].apply(lambda x: x.cat.codes)
# df_object=df_object.apply(lambda x: x.cat.codes)
# df_object =

In [32]:
def say_cross_validate_rezult(cv_rez):
    print('Алгоритм классификации: {}'.format(type(cv_rez['estimator'][0][1]).__name__))
    print('     Количество фолдов: {}'.format(len(cv_rez['estimator'])))

    print('  Время обучения фолда: {:.2f}'.format(cv_rez['fit_time'].mean()))
    print('               ROC-AUC: {:.4f}'.format(cv_rez['test_roc_auc'].mean()))
    print('             precision: {:.4f}'.format(cv_rez['test_precision'].mean()))
    print('                recall: {:.4f}'.format(cv_rez['test_recall'].mean()))
    print('               Мера f1: {:.4f}'.format(cv_rez['test_f1'].mean()))
    print()
    
def cross_validate_rezult(cv_rez):
    return [
        type(cv_rez['estimator'][0]).__name__, 
#         type(cv_rez['estimator'][0][1]).__name__,
        cv_rez['test_roc_auc'].mean(),
        cv_rez['test_precision'].mean(),
        cv_rez['test_recall'].mean(),
        cv_rez['test_f1'].mean(),
        cv_rez['test_roc_auc'].std(),
        cv_rez['test_precision'].std(),
        cv_rez['test_recall'].std(),
        cv_rez['test_f1'].std(),
        len(cv_rez['estimator']),
        cv_rez['fit_time'].mean()            
    ]

In [ ]:
# Сделаем копию данных и заменим столюцы строк (тип object) на их числовые значения (выполним LabelEncoder)
# преобразование в лоб всех столбцов по OneHotEncoder без предварительной подготовки
# данных приводит к не подъёмному DataFrame
df_object = df_tmp.copy()
columns = df_object.select_dtypes(['object']).columns
df_object[columns] = df_object[columns].astype('category')
df_object[columns] = df_object[columns].apply(lambda x: x.cat.codes)
df_object.info()
# Приведем к виду 0,1 
# df_object.loc[df_object['Label'] == -1, 'Label'] = 0

In [33]:
# X = df_test.iloc[:,:-1]
# si = SimpleImputer(strategy='most_frequent')
si = SimpleImputer(strategy='mean')
X = si.fit_transform(df_test)

# min_max_scaler = preprocessing.MinMaxScaler()
# X = min_max_scaler.fit_transform(X)

In [34]:
# X = df_object.iloc[:,:-1]
y = df_data['Label']
#  most_frequent
# pipe = Pipeline([("imputer", SimpleImputer(strategy='most_frequent')),
#                       ("estimator", RandomForestClassifier(n_estimators=10))])
# RidgeClassifier
# RandomForestClassifier(n_estimators=10)
# GradientBoostingClassifier()
# accuracy = cross_val_score(pipe, X, y, cv=3, scoring='accuracy' )

# accuracy = cross_val_score(pipe, X, y, cv=3, scoring='f1')

In [ ]:
%%time
rez = []
# cv = StratifiedKFold(n_splits=3, shuffle=True)
# rkf = RepeatedKFold(n_splits=10, n_repeats=5)
# for est in [KNeighborsClassifier(n_neighbors=5), SGDClassifier(), LogisticRegression(solver='liblinear'), RidgeClassifier(), RandomForestClassifier(n_estimators=10), GradientBoostingClassifier()]:


for est in [SGDClassifier(), 
            LinearSVC(),
            LogisticRegression( solver='lbfgs' ),
#             LogisticRegression(solver='liblinear'), 
#             RidgeClassifier(),
            RandomForestClassifier(n_estimators=10), 
            SVC(kernel='linear') 
#             GradientBoostingClassifier()
           ]:

# for est in [ RidgeClassifier(), RandomForestClassifier(n_estimators=10)]:
#     pipe = Pipeline([("imputer", SimpleImputer(strategy='most_frequent')),
#                       ("estimator", est)])

    cv_rez = cross_validate(est, X, y, cv=3, 
                          return_train_score=False, 
                          return_estimator=True,
                          verbose=0, n_jobs=4,
                          scoring=('accuracy', 'precision', 'recall', 'f1', 'roc_auc'))
    rez.append(cross_validate_rezult(cv_rez))
#     say_cross_validate_rezult(cv_rez)
# cv_rez

In [80]:
rez_df = pd.DataFrame(rez, columns=['Алгоритм классификации', 
                                   'ROC-AUC', 'precision', 'recall', 'Мера f1',
                                   'Дисп. ROC-AUC', 'Дисп.  precision', 'Дисп. recall', 'Дисп. Мера f1',
                                   'Количество фолдов', 'Время обучения фолда'])
rez_df = rez_df.round(decimals=4).sort_values(by=['ROC-AUC'])
rez_df

,Алгоритм классификации,ROC-AUC,precision,recall,Мера f1,Дисп. ROC-AUC,Дисп. precision,Дисп. recall,Дисп. Мера f1,Количество фолдов,Время обучения фолда
0,SGDClassifier,0.5571,0.4017,0.0087,0.0165,0.0117,0.1390,0.0064,0.0116,3,3.9721
1,LogisticRegression,0.5869,0.2778,0.0015,0.0029,0.0085,0.2079,0.0010,0.0020,3,2.9524
3,RandomForestClassifier,0.6934,0.6162,0.0886,0.1547,0.0041,0.0306,0.0072,0.0106,3,7.9842
2,RidgeClassifier,0.7812,0.4023,0.0995,0.1591,0.0010,0.0147,0.0161,0.0220,3,3.8818


Ограничение ser_corr_categ 1000  
GradientBoostingClassifier - 0.7961 при времени обучения фолда 339.2782 

Ограничение ser_corr_categ 800  
GradientBoostingClassifier - 0.7899 при времени обучения фолда 221.2789 

### Прогнозирование 02

In [39]:
df_test = pd.read_csv('orange_small_churn_test_data.csv')
df_test.rename(columns={"ID": "Id"}, inplace=True)

In [44]:
df_test['Var67'].unique()

array([nan,  0.])

In [81]:
%%time
df_test = pd.read_csv('orange_small_churn_test_data.csv')
df_test.rename(columns={"ID": "Id"}, inplace=True)

df_test = df_test.drop(columns=columns_nan)
df_test['Var67'].fillna(value=-1, inplace=True)
df_test['Var118'].fillna(value=-1, inplace=True)

df_categ = df_test[columns_categ_used].copy()
# теперь в категориальных полях оставим только коррелирующие значения, заменив остальные на 
for c in df_categ.columns:
    lvals = ser_corr_categ.loc[ser_corr_categ.Var == c].Val.values
    df_categ[c] = df_categ[c].apply(lambda x: x if x in lvals else 'Другое')
    

Wall time: 2.26 s


In [82]:
# Задача в том, чтобы после get_dummies тестового набора наложить полученные данные на
# df_dummies
# df_dummies.shape
# df_categ.shape
empty_arr = np.zeros((df_categ.shape[0], df_dummies.shape[1]))
df_dummies_test = pd.DataFrame(empty_arr, columns=df_dummies.columns)
df_dummies_test.info()
df_dummies_tmp =pd.get_dummies(df_categ)
for c in df_dummies_tmp.columns:
    if c in df_dummies.columns:
        df_dummies_test[c] = df_dummies_tmp[c]
        


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 1032 entries, Var192_1JGTmBQZiT to Var229_Другое
dtypes: float64(1032)
memory usage: 78.7 MB


In [83]:
df_test_X = pd.concat([df_test[columns_val_used], df_dummies_test], axis=1)

# si = SimpleImputer(strategy='most_frequent')
si = SimpleImputer(strategy='mean')

X_test = si.fit_transform(df_test_X)
# min_max_scaler = preprocessing.MinMaxScaler()
# X_test = min_max_scaler.fit_transform (X_test)

In [84]:
%%time

# est = GradientBoostingClassifier()
est = RidgeClassifier()
est.fit(X, y)

Wall time: 1.81 s


C:\ProgramData\Anaconda2\envs\newpy\lib\site-packages\sklearn\linear_model\ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.97755e-18): result may not be accurate.
  overwrite_a=True).T


In [85]:
y_test = est.predict_proba(X_test)
df_test['result'] = y_test[:,1]
df_test[['Id', 'result']].round(2).to_csv('dminik6_part2_w3_02.csv', index=False)

AttributeError: 'RidgeClassifier' object has no attribute 'predict_proba'

### Прогнозирование 01

In [ ]:
df_test = pd.read_csv('orange_small_churn_test_data.csv')
df_test.rename(columns={"ID": "Id"}, inplace=True)


columns = df_test.select_dtypes(['object']).columns
df_test[columns] = df_test[columns].astype('category')
df_test[columns] = df_test[columns].apply(lambda x: x.cat.codes)
df_test.info()

In [ ]:
X_test = df_test.iloc[:,:-1]
si = SimpleImputer(strategy='mean')

X_test = si.fit_transform(X_test)
min_max_scaler = preprocessing.MinMaxScaler()
X_test = min_max_scaler.fit_transform(X_test)


In [ ]:
est = GradientBoostingClassifier()
est.fit(X, y)

In [ ]:
y_test = est.predict_proba(X_test)
# y_pred = est.predict(X_test)

# y_test[:,1]
df_test['result'] = y_test[:,1]
df_test[['Id', 'result']].round(2).to_csv('dminik6_part2_w3_00.csv', index=False)
# df_test[['Id', 'y']].to_csv('dminik6_part2_w3_00.csv', index=False)

In [ ]:
y_test

In [ ]:
# rez_df.to_csv('01-С нормализацией X.csv')
rez_df

In [ ]:
say_cross_validate_rezult(cv_rez)

Алгоритм классификации: RidgeClassifier  
     Количество фолдов: 3  
  Время обучения фолда: 0.67  
               ROC-AUC: 0.6711  
             precision: 0.0000  
                recall: 0.0000  
               Мера f1: 0.0000  

Алгоритм классификации: RandomForestClassifier  
     Количество фолдов: 3  
  Время обучения фолда: 1.51  
               ROC-AUC: 0.5717  
             precision: 0.1500  
                recall: 0.0613  
               Мера f1: 0.0421  

Алгоритм классификации: GradientBoostingClassifier  
     Количество фолдов: 3  
  Время обучения фолда: 17.48  
               ROC-AUC: 0.7060  
             precision: 0.2011  
                recall: 0.3094  
               Мера f1: 0.0627  


# Выбор переменных

In [ ]:
df_data = pd.read_csv('orange_small_churn_train_data.csv')
df_data.rename(columns={"labels": "Label"}, inplace=True)
# категориальные переменные с наибольшим коэффициентом корреляции
cat_var = ['Var199','Var192','Var216', 'Var206','Var212','Var205','Var228','Var193','Var207','Var227', 'Label']
df_tmp=df_data[np.isfinite(df_data['Label'])].dropna(axis=1, how='all')
df_cat_var = df_tmp[cat_var].copy()
# df_cat_var.info()

In [ ]:
# df = df_cat_var[['Var192', 'Label']]
# Категории, которых нет в положительном результате удалим
categories_allowed = {}
df_positive = df_cat_var[df_cat_var['Label'] > 0 ].index
for col in cat_var:
    allowed_vals =  df_cat_var[df_cat_var.index.isin(df_positive)][col].value_counts().index
    categories_allowed[col] = 
    df_cat_var.loc[~df_cat_var[col].isin(allowed_vals), col] = np.nan
# df_positive
# allowed_vals = df[df_positive].Var192.value_counts().index
# # .groupby(['Var192'])

# df_cat_var.loc[~df['Var192'].isin(allowed_vals), 'Var192'] = np.nan

In [ ]:
# df_cat_var['Var192'].value_counts()

Для категориальных предикторов с большим количеством категорий объединим редкие категории в одну ("other").

In [ ]:
# val_count = df_cat_var['Var199'].value_counts()
# Блок позволяет подобрать порог отсечения мелких категорий (проценты от max к)
# исходя из желаемого количества столбцов
n_columns = 0.
for col in cat_var:
    val_count = df_cat_var[col].value_counts()
    new_cols = val_count.shape[0]
    if new_cols > 100:
        threshold = val_count.max() * 0.05
        new_cols = val_count[ val_count > threshold ].shape[0] 
        print('{} {:>4.0f} {:>4.0f}'.format(col, val_count.shape[0], new_cols ))
        to_remove = val_count[val_count <= threshold].index
        df_cat_var[col].replace(to_remove, 'other', inplace=True)
    
    n_columns += new_cols 
# val_count.shape[0]
print('Columns:', n_columns)

In [ ]:
df_cat_var.drop('Label', axis=1, inplace=True)

In [ ]:
# df_cat_var.info()

In [ ]:
# df_cat_var = df_cat_var.astype('category')
# df_cat_var = df_cat_var.apply(lambda x: x.cat.codes)
df_cat_var_dummies = pd.get_dummies(df_cat_var)

In [ ]:
df_cat_var_dummies.info()

In [ ]:
# df_tmp.select_dtypes(['object']).columns

In [ ]:
num_var = ['Var131','Var69','Var53','Var177','Var180','Var114','Var111','Var153',
           'Var154','Var59','Var124','Var10','Var33','Var139','Var38','Var5','Var182','Var36',
           'Var92']

df_test = pd.concat([df_tmp[num_var].fillna(-1), df_cat_var_dummies, df_tmp['Label']], axis=1)

In [ ]:
# df_tmp[num_var].min()

In [ ]:
df_cat_var_dummies = None
df_test.head()

In [ ]:
df_test.columns

In [ ]:
X = df_test.iloc[:,:-1]
# si = SimpleImputer(strategy='most_frequent')
si = SimpleImputer(strategy='mean')

X = si.fit_transform(X)
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)

y = df_test.iloc[:,-1]


In [ ]:
y[np.isnan(y)]


In [ ]:
%%time
rez = []
cv = StratifiedKFold(n_splits=10, shuffle=True)
# for est in [SGDClassifier(), LogisticRegression(solver='liblinear'), RidgeClassifier(), RandomForestClassifier(n_estimators=10), GradientBoostingClassifier()]:

for est in [ LogisticRegression(solver='liblinear') ]:
#     pipe = Pipeline([("imputer", SimpleImputer(strategy='most_frequent')),
#                       ("estimator", est)])

    cv_rez = cross_validate(est, X, y, cv=cv, 
                          return_train_score=False, 
                          return_estimator=True,
                          verbose=2, n_jobs=4,
                          scoring=('accuracy', 'precision', 'recall', 'f1', 'roc_auc'))
    rez.append(cross_validate_rezult(cv_rez))


In [ ]:
rez_df = pd.DataFrame(rez, columns=['Алгоритм классификации', 
                                   'ROC-AUC', 'precision', 'recall', 'Мера f1',
                                   'Дисп. ROC-AUC', 'Дисп.  precision', 'Дисп. recall', 'Дисп. Мера f1',
                                   'Количество фолдов', 'Время обучения фолда'])
rez_df.round(decimals=4).sort_values(by=['ROC-AUC'])

# Предсказание 1

In [ ]:
df_test = pd.read_csv('orange_small_churn_test_data.csv')
df_test.rename(columns={"labels": "Label"}, inplace=True)
df_test.shape

In [ ]:
est = LogisticRegression(solver='liblinear')
# proba = cross_val_predict(est, X, y, cv=cv, method='predict_proba')
proba = cross_val_predict(est, X, y, cv=5, method='predict_proba')

proba.shape

In [ ]:
proba

In [ ]:
rez_df.round(decimals=4).sort_values(by=['ROC-AUC'])

# Проведем работы с полями, определенными Boruta

In [ ]:
df_boruta = pd.read_csv('feature_df_01.csv')
boruta_columns = df_boruta[df_boruta['rank'] == 1]['features'].values

In [ ]:
cat_columns = df_tmp[boruta_columns].select_dtypes(['object']).columns
# df_object[columns] = df_object[columns].astype('category')
# df_object[columns] = df_object[columns].apply(lambda x: x.cat.codes)
# df_object.info()
cat_columns

In [ ]:
var_columns = list(set(boruta_columns) - set(cat_columns))
df_tmp = df_data.dropna(axis=1, how='all').copy()

In [ ]:
# df_tmp[cat_columns].info()
n_columns = 0.
for col in cat_columns:
    val_count = df_tmp[col].value_counts()
    new_cols = val_count.shape[0]
    if new_cols > 100:
        threshold = 10 # val_count.max() * 0.15
        new_cols = val_count[ val_count > threshold ].shape[0] 
        print('{} {:>4.0f} {:>4.0f}'.format(col, val_count.shape[0], new_cols ))
#         to_remove = val_count[val_count <= threshold].index
#         df_tmp[col].replace(to_remove, 'other', inplace=True)
    
    n_columns += new_cols 
# val_count.shape[0]
print('Columns:', n_columns)

#### Подготовка категориальных переменных методом LabelEncoder 

In [ ]:
df_cat_var_dummies = df_tmp[cat_columns].astype('category').copy()
df_cat_var_dummies = df_cat_var_dummies.apply(lambda x: x.cat.codes)

#### Подготовка категориальных переменных методом OneHotEncoder 

In [ ]:
# df_cat_var_dummies = pd.get_dummies(df_tmp[cat_columns])

In [ ]:
# df_cat_var_dummies.info()

In [ ]:
df_test = pd.concat([df_tmp[var_columns], df_cat_var_dummies, df_tmp['Label']], axis=1)
df_cat_var_dummies = None


In [ ]:
# df_test.info()

In [ ]:
X = df_test.iloc[:,:-1]
# si = SimpleImputer(strategy='most_frequent')
si = SimpleImputer(strategy='mean')

X = si.fit_transform(X)
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)

y = df_test.iloc[:,-1]
df_test = None

In [ ]:
%%time
rez = []
cv = StratifiedKFold(n_splits=10, shuffle=True)
for est in [SGDClassifier(), LogisticRegression(solver='liblinear'), RidgeClassifier(), RandomForestClassifier(n_estimators=10), GradientBoostingClassifier()]:

# for est in [ LogisticRegression(solver='liblinear'), RandomForestClassifier(n_estimators=10), GradientBoostingClassifier()]:
#     pipe = Pipeline([("imputer", SimpleImputer(strategy='most_frequent')),
#                       ("estimator", est)])

    cv_rez = cross_validate(est, X, y, cv=cv, 
                          return_train_score=False, 
                          return_estimator=True,
                          verbose=2, n_jobs=4,
                          scoring=('accuracy', 'precision', 'recall', 'f1', 'roc_auc'))
    rez.append(cross_validate_rezult(cv_rez))

In [ ]:
rez_df = pd.DataFrame(rez, columns=['Алгоритм классификации', 
                                   'ROC-AUC', 'precision', 'recall', 'Мера f1',
                                   'Дисп. ROC-AUC', 'Дисп.  precision', 'Дисп. recall', 'Дисп. Мера f1',
                                   'Количество фолдов', 'Время обучения фолда'])
rez_df.round(decimals=4).sort_values(by=['ROC-AUC'])